# Celery

- installation : `pip install celery redis`
- the worker can be started using `celery worker -A tasks --loglevel=INFO`
- the `tasks.py` file is the following :

```python
import random
from celery import Celery

app = Celery(
    "tasks",
    broker="redis://127.0.0.1:6379/0",
    backend="redis://127.0.0.1:6379/1"
)

@app.task
def get_rand():
    return random.randint(1, 10)

@app.task
def get_sum(randoms):
    return sum(randoms)
```

## Task

### Sync

In [ ]:
from tasks import get_rand
get_rand()

### Async

In [ ]:
result = get_rand.delay()
result

In [ ]:
result.get()

## Workflow

We will not call `.delay()` directly but instead we'll create **signatures** using `.si()`.

### chain()

```
+----------+      +----------+
| get_rand +------> get_rand |
+----------+      +----------+
```

In [ ]:
from celery import chain

canvas = chain(
    get_rand.si(),
    get_rand.si()
)
canvas

In [ ]:
result = canvas.delay()
result

In [ ]:
result.get()

###  group()

```
+----------+
| get_rand +------+
+----------+      |     +---------+
                  +-----> get_sum |
+----------+      |     +---------+
| get_rand +------+
+----------+
```

In [ ]:
from celery import chain, group
from tasks import get_rand, get_sum

canvas = chain(
    group(
        get_rand.si(),
        get_rand.si(),
    ),
    get_sum.s()
)
canvas

In [ ]:
result = canvas.delay()
result

In [ ]:
result.get()